In [25]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap

from scipy.stats import binom

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [26]:
omdf = pd.read_excel(rootdir_data+'Omikron.xlsx')
omdf['Dato'] = pd.to_datetime(omdf['Dato'])
omdf['Ratio'] = omdf['AntalOmikron']/omdf['AntalTest']
omdf['Perc'] = 100 * omdf['AntalOmikron']/omdf['AntalTest']


omdf = omdf.iloc[:-2]

# Stan model definition

In [27]:
import pystan
stan_code = """

data
{ 
    int<lower=0> N; // Number of data points
    int t[N]; // Time in days since start
    int n_test[N]; // Number of tests taken among positives
    int n_omicron[N]; // Number of tests positive for omicron
} 
parameters
{ 
    real<lower=0,upper=1> theta0; // Real, un-observed freq of omicron among positives at t=0
    real<lower=0> W; // Relative fitness of omicron: Rt_omicron/Rt_delta
} 
model
{ // Priors
    theta0 ~ beta(1,100); 
    W ~ lognormal(0,2);

    // Likelihood
    for (i in 1:N) {
        real theta; theta = theta0 / ( theta0 + (1-theta0) * pow(W,-t[i]) ); 
        n_omicron[i] ~ binomial(n_test[i], theta); 
    } 
}
generated quantities
{
    real n_omicron_pred[N];
    for (i in 1:N){
        real theta; theta = theta0 / ( theta0 + (1-theta0) * pow(W,-t[i]) ); 
        n_omicron_pred[i] = binomial_rng(n_test[i], theta);
    }
}

"""

In [28]:
sm = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ed8f8b09e88e75fbea90f531ebfd5693 NOW.


# Prepare data

In [29]:
# omdfShort = omdf.iloc[:-15]
omdfShort = omdf.iloc[6:22] # As Anders' example
omdfShort = omdf.iloc[6:17]
omdfShort = omdf.iloc[6:19]
# omdfShort

In [30]:
numVals = len(omdfShort)
tRange = np.arange(numVals)
n_test = omdfShort.AntalTest.values
n_omicron = omdfShort.AntalOmikron.values

stan_data = {   "N" : numVals,
                "t" : tRange,
                "n_test" : n_test,
                "n_omicron" : n_omicron}

fig,(ax1,ax2) = plt.subplots(2,1,figsize=(10,7))
ax1.plot(omdfShort.Dato,n_test,'*')
ax1.plot(omdfShort.Dato,n_omicron,'*')
ax2.plot(omdfShort.Dato,n_omicron/n_test,'*')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
# Make fit
fit = sm.sampling(data=stan_data,iter=100000)

In [32]:
# fit
fit.extract()

OrderedDict([('theta0',
              array([0.00237372, 0.0023115 , 0.00245478, ..., 0.00200686, 0.00198735,
                     0.00221349])),
             ('W',
              array([1.41270373, 1.41946691, 1.40997213, ..., 1.43765291, 1.4354235 ,
                     1.425059  ])),
             ('n_omicron_pred',
              array([[ 10.,  12.,  17., ..., 431., 612., 783.],
                     [  6.,  13.,  22., ..., 516., 596., 764.],
                     [  7.,  12.,  22., ..., 451., 565., 765.],
                     ...,
                     [  5.,  11.,  15., ..., 470., 575., 800.],
                     [  4.,  12.,  15., ..., 428., 604., 779.],
                     [  6.,   5.,  28., ..., 479., 571., 781.]])),
             ('lp__',
              array([-9798.08048664, -9798.72751515, -9798.2723313 , ...,
                     -9800.47421082, -9800.74905614, -9798.97124577]))])

In [33]:
# Check chains
curChains = fit.extract(permuted=True)
theta0s = curChains['theta0']
Ws = curChains['W']

fig,(ax1,ax2) = plt.subplots(2,1)
ax1.plot(theta0s,'k',linewidth=0.25)
ax2.plot(Ws,'k',linewidth=0.25)

fig,(ax1,ax2) = plt.subplots(1,2)
curHist=np.histogram(theta0s,40,density=True)
ax1.plot(curHist[1][1:],curHist[0]/np.max(curHist[0]))
curToShow = np.median(theta0s)
ax1.plot([curToShow,curToShow],[0,1],'k')
curToShow = np.quantile(theta0s,0.95)
ax1.plot([curToShow,curToShow],[0,1],'grey')
curToShow = np.quantile(theta0s,0.05)
ax1.plot([curToShow,curToShow],[0,1],'grey')
ax1.set_ylim(bottom=0)

curHist=np.histogram(Ws,40,density=True)
ax2.plot(curHist[1][1:],curHist[0]/np.max(curHist[0]))
curToShow = np.median(Ws)
ax2.plot([curToShow,curToShow],[0,1],'k')
curToShow = np.quantile(Ws,0.95)
ax2.plot([curToShow,curToShow],[0,1],'grey')
curToShow = np.quantile(Ws,0.05)
ax2.plot([curToShow,curToShow],[0,1],'grey')
ax2.set_ylim(bottom=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1.05)

In [34]:
def curModel(t,theta0,W):
    theta = theta0 / ( theta0 + (1-theta0) * np.power(W,-t) )
    return theta 


In [35]:
# # Test function 

# theta0Median = np.median(theta0s)
# WMedian = np.median(Ws)

# plt.figure() 
# plt.plot(omdfShort.Dato,curModel(tRange,theta0Median,WMedian))
# plt.plot(omdfShort.Dato,n_omicron/n_test,'.')

In [36]:
# asdf = np.arange(np.timedelta64(30,'D'))
fit.extract()

OrderedDict([('theta0',
              array([0.00237372, 0.0023115 , 0.00245478, ..., 0.00200686, 0.00198735,
                     0.00221349])),
             ('W',
              array([1.41270373, 1.41946691, 1.40997213, ..., 1.43765291, 1.4354235 ,
                     1.425059  ])),
             ('n_omicron_pred',
              array([[ 10.,  12.,  17., ..., 431., 612., 783.],
                     [  6.,  13.,  22., ..., 516., 596., 764.],
                     [  7.,  12.,  22., ..., 451., 565., 765.],
                     ...,
                     [  5.,  11.,  15., ..., 470., 575., 800.],
                     [  4.,  12.,  15., ..., 428., 604., 779.],
                     [  6.,   5.,  28., ..., 479., 571., 781.]])),
             ('lp__',
              array([-9798.08048664, -9798.72751515, -9798.2723313 , ...,
                     -9800.47421082, -9800.74905614, -9798.97124577]))])

In [37]:
# Sample from chain
numChainToSample = 500
curChains = fit.extract()
theta0s = curChains['theta0']
Ws = curChains['W']
curLen = len(theta0s)

posVals = np.arange(curLen)
sampleIndices= np.random.choice(posVals,size=(numChainToSample,))

maxNumDays = 30
tRangePred = np.arange(maxNumDays)
tRangePredDates = omdfShort.Dato.values[0] + np.arange(np.timedelta64(maxNumDays,'D'))


allYs = []
for i in sampleIndices:
    cTheta = theta0s[i]
    cW = Ws[i]

    cY = curModel(tRangePred,cTheta,cW)

    allYs.append(cY)
allYs = np.array(allYs)


In [38]:
# Plot distribution

fig,ax1 = plt.subplots()

curP = 0.975
# ax1.fill_between(tRangePredDates,100*np.quantile(allYs,curP,axis=0),100*np.quantile(allYs,1-curP,axis=0),color='b',edgecolor='none',alpha=0.2)
ax1.fill_between(tRangePredDates,100*np.quantile(allYs,curP,axis=0),100*np.quantile(allYs,1-curP,axis=0),color='xkcd:lightblue',edgecolor='none',label='95% quantiles')
curP = 0.95
# ax1.fill_between(tRangePredDates,100*np.quantile(allYs,curP,axis=0),100*np.quantile(allYs,1-curP,axis=0),color='b',edgecolor='none',alpha=0.2)
ax1.fill_between(tRangePredDates,100*np.quantile(allYs,curP,axis=0),100*np.quantile(allYs,1-curP,axis=0),color='xkcd:cyan',label='90% quantiles')
# curP = 0.9
# ax1.fill_between(tRangePredDates,100*np.quantile(allYs,curP,axis=0),100*np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
# curP = 0.75
# ax1.fill_between(tRangePredDates,100*np.quantile(allYs,curP,axis=0),100*np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
ax1.plot(tRangePredDates,100*np.median(allYs,axis=0),'xkcd:darkblue',linewidth=1,label='Median')
# ax1.plot(tRange,n_omicron/n_test,'m.',label='Data')
ax1.plot(omdf.Dato,100*omdf.Ratio,'m.',label='Data (all)')
ax1.plot(omdfShort.Dato,100*n_omicron/n_test,'m*',label='Data (used in fit)')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_ylim([0,100])

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 60
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
ax1.set_xlim(left=omdf.Dato[0])
ax1.set_xlim(right=np.datetime64('2022-01-15'))

ax1.set_yticks(np.arange(0,110,10))
ax1.grid(axis='y')

ax1.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
# # Compare number of predicted Omikron cases
# plt.figure()

# allOs = []
# curChains = fit.extract()
# theta0s = curChains['theta0']
# for i in sampleIndices:
#     allOs.append()

#     cY = curModel(tRangePred,cTheta,cW)

#     allYs.append(cY)
# allYs = np.array(allYs)
samples = np.percentile(fit.extract(permuted=True)['n_omicron_pred'], q=[5, 50, 95], axis=0)
samples = np.percentile(fit.extract(permuted=True)['n_omicron_pred'], q=[2.5, 50, 97.5], axis=0)

fig,ax1 = plt.subplots()
# plt.plot(omdfShort.Dato,samples[0])
# plt.plot(omdfShort.Dato,samples[1])
# plt.plot(omdfShort.Dato,samples[2])
# ax1.fill_between(omdfShort.Dato,samples[0],samples[2],color='grey')
# ax1.plot(omdfShort.Dato,samples[1],'k')
# ax1.plot(omdfShort.Dato,omdfShort.AntalOmikron,'*')
ax1.fill_between(omdfShort.Dato,samples[0]/omdfShort.AntalTest,samples[2]/omdfShort.AntalTest,color='grey')
curP = 0.975
ax1.fill_between(tRangePredDates,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='xkcd:lightblue',edgecolor='none',label='95% quantiles')
ax1.plot(omdfShort.Dato,samples[1]/omdfShort.AntalTest,'k')
ax1.plot(omdfShort.Dato,omdfShort.Ratio,'*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
fit

Inference for Stan model: anon_model_ed8f8b09e88e75fbea90f531ebfd5693.
4 chains, each with iter=100000; warmup=50000; thin=1; 
post-warmup draws per chain=50000, total post-warmup draws=200000.

                     mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta0             2.4e-3  1.0e-6 2.0e-4 2.0e-3 2.3e-3 2.4e-3 2.5e-3 2.8e-3  38224    1.0
W                    1.41  6.1e-5   0.01   1.39    1.4   1.41   1.42   1.43  38330    1.0
n_omicron_pred[1]    8.78  7.6e-3   3.05    3.0    7.0    9.0   11.0   15.0 161533    1.0
n_omicron_pred[2]   12.18  9.2e-3    3.6    6.0   10.0   12.0   15.0   20.0 154408    1.0
n_omicron_pred[3]    21.7    0.01   4.89   13.0   18.0   22.0   25.0   32.0 146957    1.0
n_omicron_pred[4]   32.92    0.02   6.05   22.0   29.0   33.0   37.0   45.0 141362    1.0
n_omicron_pred[5]   38.11    0.02   6.47   26.0   34.0   38.0   42.0   51.0 142982    1.0
n_omicron_pred[6]   53.42    0.02   7.64   39.0   48.0   53.0   58.0   69.0 146309   

# Stan tests

In [16]:
stanModel = """
data {
int<lower=0> N;
vector[N] x;
vector[N] y;
}
parameters {
real alpha;
real beta;
real<lower=0> sigma;
}
model {
y ~ normal(alpha + beta * x, sigma);
}
"""

In [17]:
# posterior = pystan.build(stanModel,data=stan_data)
sm = pystan.StanModel(model_code=stanModel)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_794f50d604c29189cf9b2de560c3c716 NOW.


In [18]:
a = 0.4
b = 2.5
s = 10

# x = np.arange(0,20)
numVals = 30
x = np.linspace(0,20,numVals)
curNoise = np.random.normal(0,s,x.shape)
yTrue = a + b * x 
yNoise = yTrue + curNoise

plt.figure(figsize=(10,5))
plt.plot(x,yTrue)
plt.plot(x,yNoise,'*')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
stan_data = {   "N" : numVals,
                "x" : x,
                "y" : yNoise}
stan_data

{'N': 30,
 'x': array([ 0.        ,  0.68965517,  1.37931034,  2.06896552,  2.75862069,
         3.44827586,  4.13793103,  4.82758621,  5.51724138,  6.20689655,
         6.89655172,  7.5862069 ,  8.27586207,  8.96551724,  9.65517241,
        10.34482759, 11.03448276, 11.72413793, 12.4137931 , 13.10344828,
        13.79310345, 14.48275862, 15.17241379, 15.86206897, 16.55172414,
        17.24137931, 17.93103448, 18.62068966, 19.31034483, 20.        ]),
 'y': array([-2.23355641, -6.80531306, -4.822057  , 15.6112577 , 13.38053394,
         1.860066  , 19.20668229,  7.12533352, 22.75300751,  8.8097144 ,
        23.50866236, 25.14910418, 25.21550549, -2.12243422, 33.53974572,
        22.27101617, 41.69055702, 47.48224304, 13.18639887, 33.80432176,
        25.05394216, 37.70628592, 59.0832299 , 42.01063698, 48.4442033 ,
        45.60504719, 48.75816042, 44.01435747, 42.00425856, 71.46423039])}

In [20]:
fit = sm.sampling(data=stan_data)

In [21]:
fit2 = sm.sampling(data=stan_data, iter=10000, chains=4)

In [22]:
la = fit2.extract(permuted=True)  # return a dictionary of arrays
# mu = la['mu']

## return an array of three dimensions: iterations, chains, parameters
a = fit2.extract(permuted=False)

In [23]:
# la['alpha']

# plt.figure()

# plt.plot(la['alpha'])

numChainToSample = 5000
curLen = len(la['alpha'])
# for k in range(numChainToSample):

posVals = np.arange(curLen)
sampleIndices= np.random.choice(posVals,size=(numChainToSample,))

# plt.figure()
# for i in sampleIndices:
#     cA = la['alpha'][i]
#     cB = la['beta'][i]

#     cY = cA + cB * x
#     plt.plot(x,cY,'k',alpha=0.1)

allYs = []
for i in sampleIndices:
    cA = la['alpha'][i]
    cB = la['beta'][i]

    cY = cA + cB * x

    allYs.append(cY)
allYs = np.array(allYs)

# np.mean(allYs,axis=0).shape
# np.mean(allYs,axis=0)

# print(np.quantile(allYs,.50,axis=0))
# print(np.median(allYs,axis=0))

plt.figure()
# plt.plot(x,np.mean(allYs,axis=0),'k')
# plt.plot(x,np.median(allYs,axis=0),'k')

# plt.plot(x,np.quantile(allYs,0.9,axis=0),color='grey')
# plt.plot(x,np.quantile(allYs,1-0.9,axis=0),color='grey')

curP = 0.99
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
curP = 0.95
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
curP = 0.9
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
curP = 0.75
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
plt.plot(x,np.median(allYs,axis=0),'k')
plt.plot(x,yTrue,'b*:')
plt.plot(x,yNoise,'m.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# pystan.extract_log_lik(fit2)
# pystan.__dict__
asdf = fit.extract()['lp__']
plt.figure()
plt.plot(asdf)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …